In [ ]:
import sys
import os
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
if os.path.exists('./core'):
    os.remove('./core')

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
    sys.path.append('../symbolicregression/')

# Install

In [ ]:
%%capture
if IN_COLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !git clone https://github.com/facebookresearch/symbolicregression symbolic
    %mv ./symbolic/* ./
    %rm -rf symbolic
    !conda env create --name symbolic regression --file=environment.yml
    !conda init
    !activate symbolic
    !pip install git+https://github.com/pakamienny/sympytorch
    !conda install -c conda-forge pysr
    !conda install -c conda-forge julia
    !python3 -m pysr install

In [ ]:
import pysr
from pysr import PySRRegressor
pysr.julia_helpers.init_julia()

In [ ]:
from utils import *
set_seed(42)

In [ ]:
## Tranfsormer Model

In [ ]:
transformer=get_transformer_model()

# Known Potential experiment

In [ ]:
n_samples=30000-1
df = get_robot_data(from_sample=False,from_grid=True,
                    n_samples=n_samples,
                    parameter_static_X=[4,2.5],parameter_static_Y=[4,2.5]).astype('float32')
df = df.reset_index().drop('index',1)
n_train = int(len(df)*0.9)
train_index = np.random.choice(df.index,n_train)
test_index = np.array([i for i in df.index if i not in train_index])
robot_df_train = df.iloc[train_index]
robot_df_test = df.iloc[test_index]


In [ ]:
P = robot_df_train['x,y'.split(',')].values
j2 = robot_df_train['j2'].values

In [ ]:
t = perf_counter()
transformer.fit(P,j2)
t = perf_counter() - t

In [ ]:
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/",'x_0':'x','x_1':'y'}
transformer_str = transformer.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    transformer_str = transformer_str.replace(op,replace_op)

In [ ]:
robot_model_transformer = round_expr(sp.parse_expr(transformer_str),5)

In [ ]:
j2_approx_tranfromer=sp.lambdify(['x','y'],robot_model_transformer,"numpy")

In [ ]:
str(robot_model_transformer)

In [ ]:
try:
    approx_train = j2_approx_tranfromer(robot_df_train['x'].values,robot_df_train['y'].values)
    train_err = mse(approx_train,robot_df_train['j2'])
    approx_test = j2_approx_tranfromer(robot_df_test['x'].values,robot_df_test['y'].values)
    test_err = mse(approx_test,robot_df_test['j2'])
    print(train_err,test_err)
except Exception as e:
    print(e)

In [ ]:
fig = plt.figure(figsize=(10,10))
arr = robot_df_train.values
ax = fig.add_subplot(111, projection='3d')
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer')
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2')

arr = robot_df_test.values

ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer (test)',marker='x')
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',marker='x')

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
nproc = 64

In [ ]:

robot_model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    procs = nproc,
    timeout_in_seconds  = 100,
    binary_operators=["+",'-', "*",],
    temp_equation_file=True,
    delete_tempfiles=True,
    unary_operators=[
        "atan",
        "exp",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

In [ ]:
t2 = perf_counter()
robot_model.fit(P[:],j2[:])
t2 = perf_counter()-t2

In [ ]:
print(t2)

In [ ]:
if os.path.exists('./core'):
    os.remove('./core')

In [ ]:
save_eq = False

In [ ]:
best_eq_id = robot_model.equations_['loss'].argmin()

In [ ]:
if save_eq:
    robot_model.equations_.to_csv('./'+'robot_model.equations_.csv'.replace('.','_'))

In [ ]:
robot_model_pysr = robot_model.equations_.iloc[best_eq_id]['equation']
robot_model_pysr = str(round_expr(sp.parse_expr(robot_model_pysr),5)).replace('x0','x').replace('x1','y').replace('atan','arctan')

In [ ]:
str(robot_model_pysr)

In [ ]:
j2_approx_pysr=sp.lambdify(['x','y'],robot_model_pysr, modules=["numpy"])

In [ ]:
try:
    approx_train = j2_approx_pysr(robot_df_train['x'].values,robot_df_train['y'].values)
    train_err = mse(approx_train,robot_df_train['j2'])
    approx_test = j2_approx_pysr(robot_df_test['x'].values,robot_df_test['y'].values)
    test_err = mse(approx_test,robot_df_test['j2'])
    print(train_err,test_err)
except Exception as e:
    print(e)

In [ ]:
np.arctan

In [ ]:
params = {'text.usetex' : False,
          'font.size' : 36,
          'legend.fancybox':True,
          'legend.loc' : 'best',

          'legend.framealpha': 0.9,
          "legend.fontsize" : 21,
         }
plt.rcParams.update(params)

In [ ]:
fig = plt.figure(figsize=(15,15))
arr = robot_df_train.values
s=90
ax = fig.add_subplot(111, projection='3d')
plt.xlabel("x")
plt.ylabel("y")
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_pysr(arr[:,1],arr[:,2]),color='magenta',label='pysr',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',s=s)

arr = robot_df_test.values

ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',marker='x',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_pysr(arr[:,1],arr[:,2]),color='magenta',marker='x',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',marker='x',s=s)
ax.scatter(0,0,color='black',marker='o',label='train',s=90)
ax.scatter(0,0,color='black',marker='x',label='test',s=90)

plt.legend()
plt.tight_layout()

## equations

In [ ]:
print(robot_model_transformer)

In [ ]:
print(robot_model_pysr)